<a href="https://colab.research.google.com/github/cherryMonth/Neural-Networks/blob/master/tensorflow/Pre_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! wget http://download.tensorflow.org/models/resnet_v1_50_2016_08_28.tar.gz

--2019-01-30 02:56:10--  http://download.tensorflow.org/models/resnet_v1_50_2016_08_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.197.128, 2607:f8b0:400e:c03::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.197.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95073259 (91M) [application/x-tar]
Saving to: ‘resnet_v1_50_2016_08_28.tar.gz’

resnet_v1_50_2016_0 100%[===================>]  90.67M   184MB/s    in 0.5s    

2019-01-30 02:56:11 (184 MB/s) - ‘resnet_v1_50_2016_08_28.tar.gz’ saved [95073259/95073259]



In [0]:
! tar zxvf resnet_v1_50_2016_08_28.tar.gz

resnet_v1_50.ckpt


In [0]:
! mkdir -p new_model

In [0]:
import tensorflow as tf
import keras
from tensorflow.contrib.slim import nets
import numpy as np

slim = tf.contrib.slim
from keras.datasets.cifar10 import load_data

def color_preprocessing(x_train, x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:, :, :, i] = (x_train[:, :, :, i] - mean[i]) / std[i]
        x_test[:, :, :, i] = (x_test[:, :, :, i] - mean[i]) / std[i]
    return x_train, x_test

class ShowProcess():
    """
    显示处理进度的类
    调用该类相关函数即可实现处理进度的显示
    """
    i = 0  # 当前的处理进度
    max_steps = 0  # 总共需要处理的次数
    max_arrow = 50  # 进度条的长度
    infoDone = 'done'

    # 初始化函数，需要知道总共的处理次数
    def __init__(self, max_steps):
        self.max_steps = max_steps
        self.i = 0

    # 显示函数，根据当前的处理进度i显示进度
    # 效果为[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
    def show_process(self, info, i=None):
        if i is not None:
            self.i = i
        else:
            self.i += 1
        num_arrow = int(self.i * self.max_arrow / self.max_steps)  # 计算显示多少个'>'
        num_line = self.max_arrow - num_arrow  # 计算显示多少个'-'
        percent = self.i * 100.0 / self.max_steps  # 计算完成进度，格式为xx.xx%
        process_bar = '\r[' + '>' * num_arrow + '-' * num_line + ']' \
                      + '%.2f' % percent + '%  ' + info  # 带输出的字符串，'\r'表示不换行回到最左边
        print(process_bar, end='')  # 这两句打印字符到终端
        if self.i > self.max_steps:
            self.close()

    def close(self):
        print("\n")  # 训练完一行记录之后跳转到下一行
        self.i = 0

n_epochs = 200
batch_size = 128
iterations = 50000 // batch_size + 1

(x_train, y_train), (x_test, y_test) = load_data()  # 50000, 32,32,3
y_train = np.array(keras.utils.to_categorical(y_train, 10).tolist())
y_test = np.array(keras.utils.to_categorical(y_test, 10).tolist())
x_train, x_test = color_preprocessing(x_train, x_test)
x = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool, shape=[])
p = ShowProcess(iterations)

# Specify which gpu to be used

resnet_model_path = 'resnet_v1_50.ckpt'  # Path to the pretrained model
model_save_path = 'new_model/model'  # 命名规则是路径名 + 模型名称

is_training = tf.placeholder(tf.bool, name='is_training')

with slim.arg_scope(nets.resnet_v1.resnet_arg_scope()):
    net, endpoints = nets.resnet_v1.resnet_v1_50(x, num_classes=None, is_training=is_training)
    
net = slim.conv2d(net, 10, [1, 1], activation_fn=None, normalizer_fn=None, scope='fc')
net = tf.squeeze(net, [1, 2], name="SpatialSqueeze")

Using TensorFlow backend.


In [0]:
checkpoint_exclude_scopes = 'Logits'
exclusions = None
if checkpoint_exclude_scopes:
  exclusions = [
      scope.strip() for scope in checkpoint_exclude_scopes.split(',')]
variables_to_restore = []
for var in slim.get_model_variables():
  excluded = False
  for exclusion in exclusions:
      if var.op.name.startswith(exclusion):
          excluded = True
  if not excluded and var.op.name not in ['fc/weights', 'fc/biases']:  # 下载的模型不含有这两个参数，很奇怪,所以如果要使用模型，就需要把这两个参赛去掉
      variables_to_restore.append(var)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=net))

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(net, 1), tf.argmax(y, 1)), dtype=tf.float32))

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.control_dependencies(update_ops):
  optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(x_train)

init = tf.global_variables_initializer()

saver_restore = tf.train.Saver(var_list=variables_to_restore)

# config = tf.ConfigProto(allow_soft_placement = True) 
# config.gpu_options.per_process_gpu_memory_fraction = 0.95

with tf.Session() as sess:
    sess.run(init)

    # Load the pretrained checkpoint file xxx.ckpt
    saver_restore.restore(sess, resnet_model_path)

    for epoch_i in range(n_epochs):
      result = None
      average_result = 0.0
      average_loss = 0.0
      count = 0
      k = 0
      for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=batch_size):
          result = sess.run([optimizer, accuracy, loss], feed_dict={x: x_batch, y: y_batch, is_training: True})
          average_result += result[1]
          average_loss += result[2]
          p.show_process('epoch: {}, step: {}, loss: {:.3f}, acc: {:.2f}'.format(epoch_i + 1,
                                                                                 k + 1, result[2], result[1], 3), k)
          k += 1
          if k == iterations: break

      average_result /= iterations
      average_loss /= iterations
      if epoch_i % 10 == 0:
          saver = tf.train.Saver(tf.global_variables())
          save_path = saver.save(sess, model_save_path)
      index = np.random.randint(10000, size=(1000,))
      result = sess.run([accuracy, loss], feed_dict={x: x_test[index], y: y_test[index], is_training: False})
      info = "epoch: {}, step:{}, average-loss: {:.3f}, average-acc: {:.2f}, val_loss: {:.3f}, val_acc: {:.2f}".format(
          epoch_i,
          k + 1,
          average_loss,
          average_result,
          result[1],
          result[0])
      p.show_process(info, iterations)
      p.close()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

INFO:tensorflow:Restoring parameters from resnet_v1_50.ckpt
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%  epoch: 0, step:392, average-loss: 1.545, average-acc: 0.49, val_loss: 3.531, val_acc: 0.08

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%  epoch: 1, step:392, average-loss: 0.948, average-acc: 0.67, val_loss: 3.669, val_acc: 0.09

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%  epoch: 2, step:392, average-loss: 0.790, average-acc: 0.72, val_loss: 3.699, val_acc: 0.12

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%  epoch: 3, step:392, average-loss: 0.692, average-acc: 0.76, val_loss: 2.749, val_acc: 0.24

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%  epoch: 4, step:392, average-loss: 0.628, average-acc: 0.78, val_loss: 1.562,